1. first reproduced the high scores
2. try to use normalization accross samples and see if this can help increase the score

In [28]:
import numpy as np
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier


In [38]:
def convert_onehot(y_ex):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y_ex.astype('category'))
#     # binary encode
#     onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
#     integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#     y_coded = onehot_encoder.fit_transform(integer_encoded)
    
    return (label_encoder, integer_encoded)

In [45]:
def make_prediction_kfold(test, data, y):
    
    le, target = convert_onehot(y.surface)
    target = target.ravel()
    print (np.shape(target))

    n_class = 9
    
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=20)
    predicted = np.zeros((test.shape[0],n_class))
    measured= np.zeros((data.shape[0]))
    score = 0
    
    print (np.shape(test))
    print (np.shape(data))

    for times, (trn_idx, val_idx) in enumerate(folds.split(data.values,target)):
        model = RandomForestClassifier(n_estimators=500, n_jobs = -1)
        model.fit(data.iloc[trn_idx],target[trn_idx])
        measured[val_idx] = model.predict(data.iloc[val_idx])
        predicted += model.predict_proba(test)/folds.n_splits
        score += model.score(data.iloc[val_idx],target[val_idx])
        print("Fold: {} score: {}".format(times,model.score(data.iloc[val_idx],target[val_idx])))
        gc.collect()
    
    result=le.inverse_transform(predicted.argmax(axis=1))
    result_pd = pd.DataFrame({'series_id': list(range(len(result))), 'surface':result})
                              
    return (result_pd)
    
    

In [11]:
X_test_all = pd.read_csv('New_data\X_test_selected_features.csv')
X_train_all = pd.read_csv('New_data\X_train_selected_features.csv')
y_train_all = pd.read_csv('New_data\Train_y.csv')

In [12]:
X_test_all.head()

,id,"orientation_Z__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10",linear_acceleration_Z__partial_autocorrelation__lag_4,"orientation_W__agg_linear_trend__f_agg_""var""__chunk_len_5__attr_""rvalue""","orientation_W__fft_coefficient__coeff_21__attr_""real""","orientation_Z__fft_coefficient__coeff_27__attr_""real""",angular_velocity_X__longest_strike_above_mean,"orientation_X__fft_coefficient__coeff_19__attr_""imag""","angle_Y__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_20","orientation_Y__fft_coefficient__coeff_53__attr_""abs""",...,angle_Z__abs_energy,linear_acceleration_X__partial_autocorrelation__lag_6,orientation_W__abs_energy,"angular_velocity_X__fft_aggregated__aggtype_""centroid""","orientation_X__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_5","angle_Y__fft_coefficient__coeff_21__attr_""abs""",orientation_W__partial_autocorrelation__lag_8,"angle_X__fft_coefficient__coeff_36__attr_""abs""","orientation_Y__fft_coefficient__coeff_57__attr_""real""","orientation_W__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""stderr"""
0,0,-0.243516,-0.128970,-0.271917,-0.000634,-0.000660,5.0,0.000481,-0.005533,0.000198,...,1223.315786,0.070472,0.001275,22.620098,-0.000007,0.001437,-0.477185,0.000207,0.000088,0.000006
1,1,0.070733,-0.117502,-0.332469,-0.001564,0.001028,18.0,-0.002241,-0.004646,0.002857,...,59.034651,-0.173276,2.543646,16.680664,0.000177,0.000281,-0.462603,0.002491,0.002807,0.000015
2,2,0.238500,0.016427,-0.225892,0.001418,-0.000237,5.0,-0.003672,-0.005516,0.000322,...,912.033423,0.002008,0.230062,22.401308,0.001558,0.001431,-0.406308,0.001419,0.000596,0.000008
3,3,0.241586,0.200422,0.131696,-0.000491,0.000196,4.0,0.004283,-0.003659,0.000342,...,1009.975832,0.320353,0.041927,20.748559,-0.000111,0.000577,-0.013068,0.000638,0.000293,0.000002
4,4,0.235623,-0.031627,-0.590452,0.002659,0.000978,29.0,-0.032946,-0.002930,0.004485,...,879.387663,0.010791,0.248844,14.112171,0.002739,0.000528,-0.015644,0.000534,0.004363,0.000002


In [46]:
result = make_prediction_kfold(X_test_all.drop(['id'], axis = 1), X_train_all.drop(['id'], axis = 1), y)

(4021,)
(3816, 4989)
(4021, 4989)
Fold: 0 score: 0.9187192118226601
Fold: 1 score: 0.8793103448275862
Fold: 2 score: 0.9158415841584159
Fold: 3 score: 0.9280397022332506
Fold: 4 score: 0.8980099502487562
Fold: 5 score: 0.9104477611940298
Fold: 6 score: 0.8805970149253731
Fold: 7 score: 0.905
Fold: 8 score: 0.9122807017543859
Fold: 9 score: 0.9219143576826196


In [48]:
result.to_csv(r'New_data\results\RF_500_check_20190404.csv', index=False)

In [49]:
# yes, the resutl is the same 0.68

In [50]:
from sklearn.preprocessing import normalize

In [65]:
def make_prediction_kfold_norm(test, data, y):
    
    le, target = convert_onehot(y.surface)
    target = target.ravel()
    print (np.shape(target))

    n_class = 9
    
    ## normalize the data
    data = normalize(data)
    test =normalize(test)
    
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=20)
    predicted = np.zeros((test.shape[0],n_class))
    measured= np.zeros((data.shape[0]))
    score = 0
    
    print (np.shape(test))
    print (np.shape(data))

    for times, (trn_idx, val_idx) in enumerate(folds.split(data,target)):
        model = RandomForestClassifier(n_estimators=500, n_jobs = -1)
        model.fit(data[trn_idx,:],target[trn_idx])
        measured[val_idx] = model.predict(data[val_idx,:])
        predicted += model.predict_proba(test)/folds.n_splits
        score += model.score(data[val_idx,:],target[val_idx])
        print("Fold: {} score: {}".format(times,model.score(data[val_idx,:],target[val_idx])))
        gc.collect()
    
    result=le.inverse_transform(predicted.argmax(axis=1))
    result_pd = pd.DataFrame({'series_id': list(range(len(result))), 'surface':result})
                              
    return (result_pd)
    

In [66]:
result_norm = make_prediction_kfold_norm(X_test_all.drop(['id'], axis = 1), X_train_all.drop(['id'], axis = 1), y)

(4021,)
(3816, 4989)
(4021, 4989)
Fold: 0 score: 0.7536945812807881
Fold: 1 score: 0.687192118226601
Fold: 2 score: 0.7128712871287128
Fold: 3 score: 0.7444168734491315
Fold: 4 score: 0.6890547263681592
Fold: 5 score: 0.7164179104477612
Fold: 6 score: 0.6940298507462687
Fold: 7 score: 0.71
Fold: 8 score: 0.7042606516290727
Fold: 9 score: 0.7581863979848866


In [62]:
result_norm.to_csv(r'New_data\results\RF_500_check_norm_data_20190404.csv', index=False)
# score 0.52

## use the whole data set in training

In [69]:
def make_prediction_kfold_norm_whole_training_dataset(test, data, y):
    
    le, target = convert_onehot(y.surface)
    target = target.ravel()
    print (np.shape(target))

    n_class = 9
    
    ## normalize the data
    data = normalize(data)
    test =normalize(test)
    
    print (np.shape(test))
    print (np.shape(data))

    model = RandomForestClassifier(n_estimators=500, n_jobs = -1)
    model.fit(data,target)
    predicted = model.predict(test)
    
    result=le.inverse_transform(predicted)
    result_pd = pd.DataFrame({'series_id': list(range(len(result))), 'surface':result})
                              
    return (result_pd)
    

In [70]:
result_norm_whole = make_prediction_kfold_norm_whole_training_dataset(X_test_all.drop(['id'], axis = 1), \
                                                                X_train_all.drop(['id'], axis = 1), y)

(4021,)
(3816, 4989)
(4021, 4989)


In [71]:
result_norm_whole.to_csv(r'New_data\results\RF_500_check_norm_data_train_whole_data_20190404.csv', index=False)
# also score 0.52